In [ ]:
!pip install langchain==0.0.284 #for chain based operations
!pip install python-dotenv==1.0.0 #for storing API keys
!pip install streamlit==1.22.0 #for creating web apps
!pip install tiktoken==0.4.0  #for tokenization
!pip install faiss-cpu==1.7.4 #similarity search
!pip install protobuf==3.20.3 #for serializing structured data
!pip install chardet==5.0.0
!pip install langchain-community==0.0.3
!pip install transformers==4.34.0
!pip install torch==2.0.1

In [ ]:
#initializing LLM using GooglePalm class from langchain.llms module
from langchain.llms import GooglePalm
api_key="YOUR GOOGLE API KEY"
llm=GooglePalm(google_api_key=api_key, temperature=0.7)

In [ ]:
#load data from a CSV file using the CSVLoader from langchain.document_loaders.csv_loader
from langchain.document_loaders.csv_loader import CSVLoader
import chardet #handling character encoding detection
with open("/content/data.csv", 'rb') as f:
  result=chardet.detect(f.read(10000))
  detected_encoding=result['encoding'] # analyzes 1st 10000 bytes encodings
  #print(detected_encoding)

loader=CSVLoader(file_path="/content/data.csv", source_column='Question', encoding=detected_encoding)
data=loader.load() #load data from CSV to variable data

In [ ]:
!pip install sentence_transformers  #designed for computing dense vector representations of sentences and paragraphs

In [ ]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name="BAAI/bge-large-en"                  #large English model optimized for embeddings.
model_kwargs={"device": "cpu"}                  #to run the model on the CPU.
encode_kwargs={"normalize_embeddings": True}    #ensures that the embeddings are normalized.
hf=HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs) # initializes an instance

In [ ]:
#to create a vector database from a set of documents using embeddings generated by the Hugging Face model
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
embeddings=HuggingFaceBgeEmbeddings()   #initializes an instance with default parameters
vector_db=FAISS.from_documents(documents=data, embedding=embeddings)

In [ ]:
#retrieve relevant data related to the query
retriever = vector_db.as_retriever()
r_docs=retriever.get_relevant_documents("How do I know if I have covid-19?")
r_docs

[Document(page_content='Question: What are the symptoms of COVID-19?\nResponse: The most common symptoms of COVID-19 are fever, dry cough, and tiredness. Other symptoms may include shortness of breath, aches and pains, sore throat, and loss of taste or smell.', metadata={'source': 'What are the symptoms of COVID-19?', 'row': 0}),
 Document(page_content='Question: What should I do if I suspect a heart attack?\nResponse: If you suspect a heart attack, call emergency services immediately. Chew an aspirin if not allergic, stay calm, and sit down while waiting for medical help to arrive.', metadata={'source': 'What should I do if I suspect a heart attack?', 'row': 11}),
 Document(page_content='Question: How do I know if I have depression?\nResponse: Symptoms of depression include persistent sadness or anxiety, loss of interest in activities once enjoyed, changes in appetite or weight, trouble sleeping or sleeping too much, loss of energy, feelings of worthlessness or guilt, difficulty conce

In [ ]:
#Similarly for this
retriever = vector_db.as_retriever()
r_docs=retriever.get_relevant_documents("What are the signs of cancer?")

r_docs

[Document(page_content='Question: What are the signs of skin cancer?\nResponse: Signs of skin cancer include new growths or sores that do not heal, changes in the size, shape, or color of moles, and any unusual skin changes. Regular self-examinations and dermatologist visits are important.', metadata={'source': 'What are the signs of skin cancer?', 'row': 26}),
 Document(page_content='Question: What are the signs of pancreatic cancer?\nResponse: Signs of pancreatic cancer include jaundice, abdominal or back pain, weight loss, loss of appetite, nausea, vomiting, and new-onset diabetes. Early detection is challenging but crucial.', metadata={'source': 'What are the signs of pancreatic cancer?', 'row': 42}),
 Document(page_content='Question: How can I reduce my risk of cancer?\nResponse: Reduce your risk of cancer by avoiding tobacco, limiting alcohol consumption, maintaining a healthy diet, exercising regularly, protecting skin from excessive sun exposure, and getting regular medical scr

In [ ]:
#to set up a question answering system that retrieves relevant documents based on a query using a retriever interface
from langchain.chains import RetrievalQA
chain=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, input_key="query", return_source_documents=True) #initialize an instance

#stuff - specific or custom configuration
#return_source_documents=True - Indicates that the retrieved source documents will be returned along with the answer.


In [ ]:
print(chain("tell me how to keep good health")) #use chain object to answer a query

{'query': 'tell me how to keep good health', 'result': 'Keep good health by eating a healthy diet, exercising regularly, getting enough sleep, managing stress, and avoiding tobacco and alcohol.', 'source_documents': [Document(page_content='Question: How can I prevent heart disease?\nResponse: Prevent heart disease by maintaining a healthy diet, exercising regularly, avoiding tobacco, managing stress, keeping a healthy weight, and regularly monitoring blood pressure, cholesterol, and blood sugar levels.', metadata={'source': 'How can I prevent heart disease?', 'row': 10}), Document(page_content='Question: How can I improve my immune system?\nResponse: Improve your immune system by eating a balanced diet', metadata={'source': 'How can I improve my immune system?', 'row': 45}), Document(page_content='Question: How can I prevent the flu?\nResponse: Prevent the flu by getting an annual flu vaccine, practicing good hygiene such as frequent handwashing, avoiding close contact with sick indivi

In [ ]:
#to set up a template for generating prompts and questions in a structured format.

from langchain.prompts import PromptTemplate
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

#create an instance
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
from langchain.chains import RetrievalQA
chain=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, input_key="query", return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain("Do you have javascript course?")

{'query': 'Do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='Question: How can I improve my immune system?\nResponse: Improve your immune system by eating a balanced diet', metadata={'source': 'How can I improve my immune system?', 'row': 45}),
  Document(page_content='Question: How can I improve my sleep quality?\nResponse: Improve sleep quality by maintaining a regular sleep schedule, creating a restful environment, limiting screen time before bed, avoiding caffeine and heavy meals before bedtime, and practicing relaxation techniques.', metadata={'source': 'How can I improve my sleep quality?', 'row': 20}),
  Document(page_content='Question: How can I reduce my risk of cancer?\nResponse: Reduce your risk of cancer by avoiding tobacco, limiting alcohol consumption, maintaining a healthy diet, exercising regularly, protecting skin from excessive sun exposure, and getting regular medical screenings.', metadata={'source': 'How can I

In [ ]:
from langchain.chains import RetrievalQA
chain=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, input_key="query", return_source_documents=True)

In [ ]:
!pip install openai ipywidgets

In [ ]:
#create and manage interface elements
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

text_input = widgets.Text(
    value='',
    placeholder='Enter your query here',
    description='Query:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='primary',
    tooltip='Click to submit your query',
    icon='check'
)

result_area = widgets.Textarea(
    value='',
    placeholder='Result will be displayed here',
    description='Output:',
    disabled=True,
    layout=widgets.Layout(width='80%', height='200px'),
    style={'description_width': 'initial'}
)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=10,
    description='Processing:',
    bar_style='',
    style={'bar_color': '#00AA00'},
    layout=widgets.Layout(width='80%')
)

def simulate_processing():
    import time
    progress_bar.value = 0
    for i in range(10):
        time.sleep(0.1)
        progress_bar.value += 10

def on_button_click(b):
    user_input = text_input.value
    result_area.value = ''
    clear_output(wait=True)
    display(app_layout)
    progress_bar.value = 0
    simulate_processing()
    result = chain(user_input)
    result_area.value = f"{result}"
    progress_bar.value = 100

submit_button.on_click(on_button_click)

input_box = widgets.HBox([text_input, submit_button])
output_box = widgets.VBox([progress_bar, result_area])
app_layout = widgets.VBox([input_box, output_box])

display(app_layout)